In [2]:
!pip install telebot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 5.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

In [5]:
import json
import gzip

data = []
i = 0

with gzip.open("/content/drive/MyDrive/hahaha/names_salary_preparing.json.gz", 'rt') as f:
  if i < 5000:
    for line in f:
        i+=1
        item = json.loads(line.strip())
        data.append(item)


In [5]:
data[0]

{'vacancy_id': 'v_862116',
 'name': 'Смотритель музейный',
 'description': '<strong>Обязанности:</strong> <ul> <li>Осуществлять контроль за соблюдением посетителями правил поведения в музее</li> <li>При возникновении угрозы повреждения или кражи музейных предметов информировать руководство музея</li> </ul> <strong>Требования:</strong> <ul> <li>образование средне-специальное или высшее</li> <li>музейное дело, культурология</li> </ul> <strong>Условия:</strong> <ul> <li>Выходные: 2 дня. График с 09:00 до 18:00, по четвергам с 11:00 до 20:00</li> <li>Обед скользящий</li> </ul>',
 'compensation_from': 16500.0,
 'compensation_to': None}

In [6]:
df = pd.DataFrame(data)


In [15]:
current_vacancy = 'v_2597196'
str(vacancies.query('vacancy_id == @current_vacancy')['description'].values)

"['<p><em>Открыта вакансия <strong>Инженер по ОТ и ПБ </strong>в компании Перспектива – российская торговая FMCG (товары повседневного спроса) компания в категории безалкогольные и слабоалкогольные напитки, специализирующаяся на оптово-розничных продажах.</em></p> <p><strong>Обязанности:</strong></p> <ul> <li>Осуществление контроля за соблюдением законодательных и иных нормативно-правовых актов по охране труда</li> <li>Разработка мероприятий по предупреждению профессиональных заболеваний и несчастных случаев</li> <li>Работа с нормативно-технологической документацией (положение об организации работы по ОТ, ТБ и ПБ; положение о комиссиях по ОТ и ПБ; технологические карты по безопасной работе оборудования и т.д.)</li> <li>Ведение необходимой документации по охране труда, пожарной безопасности</li> <li>Участие в работе комиссии по проверке знаний требований охраны труда, аттестации рабочих мест</li> <li>Проведение проверок тех. состояния оборудования, механизмов, определение их соответстви

TeleBot

In [1]:
import telebot
from telebot import types

common_markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
btn1 = types.KeyboardButton("Следующая")
btn2 = types.KeyboardButton("Полное описание")
common_markup.add(btn1, btn2)

favourite_markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
btn3 = types.KeyboardButton("Следующая")
btn5 = types.KeyboardButton("Добавить в избранное")
btn4 = types.KeyboardButton("Отклик")
favourite_markup.add(btn3, btn5, btn4)


global replayer
global input_ids
global cur_state
global cnt


# Пример данных
data = {
    'u_482520': ['v_2597196', 'v_1223061', 'v_1223062'],

}
vacancies = df


TOKEN = '6781213285:AAH5n-vasxmUxSmdyZdCZOTaeRXOVWM3acM'
bot = telebot.TeleBot(TOKEN)


user_state = {}


@bot.message_handler(commands=['start'])
def start(message):
    global cur_state, markup, input_ids, cnt
    cur_state = 'autorization'
    bot.send_message(message.chat.id, 'Привет! Пожалуйста, введи свой номер:')


@bot.message_handler(content_types='text')
def message_reply(message):
    global cur_state, markup, input_ids, cnt, user_state
    user_id = message.chat.id
    if cur_state == 'autorization':

      number = message.text

      if number not in data:
          bot.send_message(user_id, 'Номер не найден. Попробуй еще раз.')
          return

      user_state[user_id] = {
          'number': number,
          'index': 0,
          'favorites': []
      }
      cur_state = 'show_vacancies'
      send_next_vacancy(user_id)

    elif message.text == "Следующая":
      state = user_state[user_id]
      state['index'] += 1
      send_next_vacancy(user_id)
    elif message.text == "Полное описание":
      state = user_state[user_id]
      current_vacancy = data[state['number']][state['index']]
      desc = str(vacancies.query('vacancy_id == @current_vacancy')['description'].values[0])
      bot.send_message(user_id, f'Описание вакансии:\n{desc}')
      bot.send_message(user_id, 'Выберите действие:', reply_markup=favourite_markup)

    elif message.text == "Добавить в избранное":
      state = user_state[user_id]
      current_vacancy = str(data[state['number']][state['index'] - 1])
      state['favorites'].append(current_vacancy)

      bot.send_message(user_id, f'Вакансия {current_vacancy} добавлена в избранное.', reply_markup = favourite_markup)

    elif message.text == "Отклик":
      bot.send_message(user_id, 'Прикиньте!! Мы только что отправили ваше великолепнейшее резюме и работодатель сразу прислал оффер!! В ответном сообщении укажите номер карты и CVC :3', reply_markup = favourite_markup)


def send_next_vacancy(user_id):
    state = user_state[user_id]
    vacancies_list = data[state['number']]
    index = state['index']

    if index >= len(vacancies_list):
        bot.send_message(user_id, 'На сегодня все, покаааааааа')
        del user_state[user_id]
        return

    vacancy_number = vacancies_list[index]
    vacancy_name = str(df.query('vacancy_id == @vacancy_number')['name'].values[0])
    salary = int(df.query('vacancy_id == @vacancy_number')['compensation_from'].values[0])
    bot.send_message(user_id, f'Вакансия: {vacancy_name}\n Зарплата: {salary}', reply_markup=common_markup)


# Запуск бота
bot.polling()


NameError: name 'df' is not defined